# Gaussian Elimination Solver in Python

Welcome

This notebook explains how I implemented Gaussian Elimination from scratch using NumPy to solve systems of linear equations of the form:

AX = B

Instead of using built-in solvers, I wrote custom functions step by step to:

- Convert the system into Row Echelon Form (REF)  
- Perform Back Substitution to solve for the variables  
- Combine everything into one complete `gaussian_elimination()` solver

This gave me a better understanding of what really happens behind the scenes when solving equations using matrices.

---

## What You Will Learn in This Notebook

This notebook walks through the logic and code for:

- `swap_rows()` – swaps two rows in a matrix  
- `get_index_first_non_zero_from_column()` – finds the first non-zero value in a column starting from a given row  
- `augmented_matrix()` – builds the augmented matrix [A | B]  
- `row_echelon_form()` – reduces the augmented matrix to Row Echelon Form  
- `back_substitution()` – performs back substitution to find the final variable values  
- `gaussian_elimination()` – ties everything together to solve the full system

---

## Why I Built This

Instead of calling `np.linalg.solve` and getting an instant result, I wanted to understand each step in the process. This notebook shows how I built up the solution by writing the row operations myself.

Let us get started and solve some linear systems step by step.


In [43]:
#importing neccesary libraries
import numpy as np
import pandas as pd

### Swap Rows: `swap_rows(M, row_index_1, row_index_2)`

In Gaussian elimination, we often need to **swap rows** to bring a non-zero pivot into the current working row. This helper function makes it easy to exchange two rows of a NumPy matrix using advanced indexing.

By using `swap_rows()`, we ensure that our algorithm remains **modular**, **reusable**, and avoids in-place side effects (since it returns a copy of the modified matrix).

Use this whenever:
- The pivot element is zero and needs to be replaced
- You need to reorder rows for better numerical stability
- You want to test matrix manipulation techniques


In [ ]:
#creating a function that swaps different rows of matrix
def swap_rows(M, row_index_1, row_index_2):
    #copy the matrix
    M = M.copy()
    # swap indexes (you use a list of indexes to index rows in Numpy)
    M[[row_index_1, row_index_2]] = M[[row_index_2, row_index_1]]
    #return the swapped matrices
    return M




### Test case Swap Rows Function

In [9]:
M = np.array([
[1, 3, 6],
[0, -5, 2],
[-4, 5, 8]
])
print(M)

[[ 1  3  6]
 [ 0 -5  2]
 [-4  5  8]]


In [13]:
swap_rows(M,0,2)

array([[-4,  5,  8],
       [ 0, -5,  2],
       [ 1,  3,  6]])

##  Get_index_first_non_zero_from_column(M, column, starting_row)

This helper function returns the index of the **first non-zero element** in a given column of a matrix, starting from a specific row downward.

### Parameters:
- `M` (*numpy.array*): The matrix to search through.
- `column` (*int*): The index of the column to look at.
- `starting_row` (*int*): The row index to start checking from (ignores rows above this).

### Returns:
- An integer: the **row index** of the first non-zero element found from `starting_row` downwards.
- If **no non-zero element** is found in that column, it returns `-1`.

### Notes:
- Uses `np.isclose` instead of `val == 0` to avoid floating-point errors when comparing with zero.

### Example:
```python
M = np.array([
    [0, 0, 0],
    [0, 5, 0],
    [0, 0, 9]
])

get_index_first_non_zero_from_column(M, column=1, starting_row=0)
# Output: 1


In [75]:

def get_index_first_non_zero_from_column(M, column, starting_row):
    """
    Find the index of the first non-zero value in a column from a given starting row.

    Parameters:
    - M (numpy.array): The matrix to search.
    - column (int): The column to search.
    - starting_row (int): The row index to start searching from.

    Returns:
    - int: Index of the first non-zero value in the column, or -1 if none found.
    """
    column_array = M[starting_row:, column]
    
    for i, val in enumerate(column_array):
        if not np.isclose(val, 0, atol=1e-5):
            return i + starting_row
    return -1



In [76]:
N = np.array([
[0, 5, -3 ,6 ,8],
[0, 6, 3, 8, 1],
[0, 0, 0, 0, 0],
[0, 0, 0 ,0 ,7],
[0, 2, 1, 0, 4]
]
)
print(N)

[[ 0  5 -3  6  8]
 [ 0  6  3  8  1]
 [ 0  0  0  0  0]
 [ 0  0  0  0  7]
 [ 0  2  1  0  4]]


In [77]:
get_index_first_non_zero_from_column(N, column = 0, starting_row = 0)

-1

In [78]:
get_index_first_non_zero_from_column(N, column = -1, starting_row = 2)

3

## Get_index_first_non_zero_from_row(M, row, augmented=False)

This helper function returns the **index of the first non-zero element** in a specified row of a matrix.

### Parameters:
- `M` (*numpy.array*): The matrix to inspect.
- `row` (*int*): The row index to check.
- `augmented` (*bool*, optional): If `True`, the function ignores the **last column** (useful when working with augmented matrices like `[A | B]`).

### Returns:
- An **integer**: the index of the first non-zero value in the row.
- If the row contains **only zeros**, returns `-1`.

### Notes:
- The function uses `np.isclose(val, 0)` instead of direct comparison (`val == 0`) to avoid floating-point inaccuracies.
- This is useful in Gaussian Elimination when identifying pivot positions in rows.

### Example:
```python
M = np.array([
    [0, 0, 0],
    [0, 0, 3],
    [1, 2, 0]
])

get_index_first_non_zero_from_row(M, row=1)
# Output: 2


In [ ]:
def get_index_first_non_zero_from_row(M, row, augmented=False):
    """
    Returns the index of the first non-zero value in a specific row.
    
    Parameters:
    - M (numpy.array): The matrix to search.
    - row (int): The row to inspect.
    - augmented (bool): If True, ignore the last column (assumed constants).

    Returns:
    - int: Index of first non-zero value, or -1 if all are zero.
    """
    M = M.copy()

    if augmented:
        M = M[:, :-1]  # Remove last column if matrix is augmented

    row_array = M[row]  # Get the specific row

    for i, val in enumerate(row_array):
        if not np.isclose(val, 0, atol=1e-5):
            return i

    return -1



In [22]:
print(N)

[[ 0  5 -3  6  8]
 [ 0  6  3  8  1]
 [ 0  0  0  0  0]
 [ 0  0  0  0  7]
 [ 0  2  1  0  4]]


In [55]:
print(f'Output for row 2: {get_index_first_non_zero_from_row(N, 2)}')
print(f'Output for row 3: {get_index_first_non_zero_from_row(N, 3)}')

Output for row 2: -1
Output for row 3: 4


##  Augmented_matrix(A, B)

This function creates an **augmented matrix** by **horizontally stacking** the coefficient matrix `A` with the constants matrix `B`.

### Parameters:
- `A` (*numpy.array*): The coefficient matrix (usually of shape *n × n*).
- `B` (*numpy.array*): The constants column vector (usually of shape *n × 1*).

### Returns:
- A new matrix `[A | B]`, obtained by horizontally stacking `A` and `B` using `np.hstack()`.

### Example:
```python
A = np.array([[1, 2], [3, 4]])
B = np.array([[5], [6]])

augmented_matrix(A, B)
# Output:
# array([[1, 2, 5],
#        [3, 4, 6]])


In [56]:
def augmented_matrix(A, B):
    """
    Create an augmented matrix by horizontally stacking two matrices A and B.

    Parameters:
    - A (numpy.array): First matrix.
    - B (numpy.array): Second matrix.

    Returns:
    - numpy.array: Augmented matrix obtained by horizontally stacking A and B.
    """
    augmented_M = np.hstack((A,B))
    return augmented_M

In [57]:
A = np.array([[1,2,3], [3,4,5], [4,5,6]])
B = np.array([[1], [5], [7]])

print(augmented_matrix(A,B))

[[1 2 3 1]
 [3 4 5 5]
 [4 5 6 7]]


## Row_echelon_form(A, B)

This function performs **Gaussian Elimination** to reduce the augmented matrix `[A | B]` into **Row Echelon Form (REF)**.

### What it Does:
- Checks if the system is **singular** (i.e. no unique solution).
- Augments the matrix `[A | B]`.
- Iteratively eliminates values below each pivot to form upper triangular structure.
- Ensures **pivot normalization** (each pivot becomes 1).
- Swaps rows if the current pivot is zero to maintain numerical stability.

### Parameters:
- `A` (*numpy.array*): Square coefficient matrix (*n × n*).
- `B` (*numpy.array*): Constants column vector (*n × 1*).

###  Returns:
- The augmented matrix `[A | B]` in **Row Echelon Form**.
- If the system is singular, returns the string `'singular system'`.

### Example:
```python
A = np.array([[2, 1, -1],
              [-3, -1, 2],
              [-2, 1, 2]])
B = np.array([[8], [-11], [-3]])

row_echelon_form(A, B)
# Output (REF):
# array([[ 1.,  0.5, -0.5,  4. ],
#        [ 0.,  1., -1.,  1. ],
#        [ 0.,  0.,  1., -2.]])


In [ ]:
def row_echelon_form(A, B):
    # Check for singular matrix
    if np.isclose(np.linalg.det(A), 0):
        return 'singular system'

    # Copy and convert to float
    A = A.astype('float64').copy()
    B = B.astype('float64').copy()

    # Augment the matrix
    M = augmented_matrix(A, B)

    num_rows = len(A)

    for row in range(num_rows):
        pivot_candidate = M[row, row]

        # If pivot is zero, look below for a row to swap
        if np.isclose(pivot_candidate, 0):
            swap_row_index = get_index_first_non_zero_from_column(M, row, row + 1)
            if swap_row_index == -1:
                continue
            M = swap_rows(M, row, swap_row_index)
            pivot = M[row, row]
        else:
            pivot = pivot_candidate

        # Normalize the pivot row
        M[row] = M[row] / pivot

        # Eliminate values below the pivot
        for j in range(row + 1, num_rows):
            value_below_pivot = M[j, row]
            M[j] = M[j] - value_below_pivot * M[row]

    return M


        

            

In [69]:
A = np.array([[1,2,3],[0,1,0], [0,0,5]])
B = np.array([[1], [2], [4]])
row_echelon_form(A,B)

array([[1. , 2. , 3. , 1. ],
       [0. , 1. , 0. , 2. ],
       [0. , 0. , 1. , 0.8]])

### Back Substitution

Once the augmented matrix has been transformed into **Row Echelon Form (REF)** using Gaussian Elimination, we use **back substitution** to compute the values of the unknown variables.

This function starts from the bottom row and works upward, eliminating the known variables from the equations above.

**What it does:**
- Assumes the matrix is in REF with leading 1s (unit pivots).
- Finds the pivot (first non-zero) in each row.
- Eliminates the corresponding variable from all rows above.
- Extracts the final solution vector from the last column of the matrix.

This is the final step to solving a linear system using Gaussian Elimination.

## Gaussian_elimination(A, B)

This is the main function that solves a system of linear equations using the **Gaussian Elimination** method.

### What It Does:
1. Combines all the steps of Gaussian elimination.
2. First converts the coefficient matrix into **Row Echelon Form (REF)**.
3. Then applies **back substitution** to find the solution vector.

### Parameters:
- A: A square matrix (n x n) containing the coefficients of the system.
- B: A column matrix (n x 1) containing the constants.

### Returns:
- A 1D numpy array with the values of the solution.
- If the matrix is singular (i.e., has no unique solution), it returns the message `'singular system'`.

### Example:
```python
A = np.array([
    [2, 1, -1],
    [-3, -1, 2],
    [-2, 1, 2]
])
B = np.array([
    [8],
    [-11],
    [-3]
])

solution = gaussian_elimination(A, B)
print(solution)
# Output: [2. 3. -1.]


In [71]:
def gaussian_elimination(A, B):
    """
    Solve a linear system represented by an augmented matrix using the Gaussian elimination method.

    Parameters:
    - A (numpy.array): Square matrix of size n x n representing the coefficients of the linear system.
    - B (numpy.array): Column matrix of size n x 1 representing the constant terms.

    Returns:
    numpy.array: The solution vector.
    """

    ### START CODE HERE ###

    # Get the matrix in row echelon form
    row_echelon_M = row_echelon_form(A, B)

    # If the system is singular, return the error message
    if isinstance(row_echelon_M, str):
        return row_echelon_M

    # Perform back substitution to get the result
    solution = back_substitution(row_echelon_M)

    ### END CODE HERE ###

    return solution


### Example: Solving a System of Equations using Gaussian Elimination

We want to solve the system of equations:



In [74]:
A = np.array([
    [2, 1, -1],
    [-3, -1, 2],
    [-2, 1, 2]
])
B = np.array([
    [8],
    [-11],
    [-3]
])
solution = gaussian_elimination(A, B)
print(solution)

[ 2.  3. -1.]
